<a align="left" href="https://ultralytics.com/yolov5" target="_blank">
<img width="1024", src="https://user-images.githubusercontent.com/26833433/125273437-35b3fc00-e30d-11eb-9079-46f313325424.png"></a>

This is the **official YOLOv5 🚀 notebook** by **Ultralytics**, and is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).
For more information please visit https://github.com/ultralytics/yolov5 and https://ultralytics.com. Thank you!

# Hyperlinks



*   https://www.youtube.com/watch?v=GRtgLlwxpc4&t=191s&ab_channel=DeepLearning
*   Install YOLOv5 on Windows and Google Colab: https://youtu.be/gDoMYuyY_qw
*   Train a Custom YOLOv5 Model to Detect Bus Numbers: https://youtu.be/5h5UtLau3Vc



# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.2-66-ge57275a Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 37.4/78.2 GB disk)


# 2. Validate
Validate a model's accuracy on [COCO](https://cocodataset.org/#home) val or test-dev datasets. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag. Note that `pycocotools` metrics may be ~1% better than the equivalent repo metrics, as is visible below, due to slight differences in mAP computation.

## COCO val
Download [COCO val 2017](https://github.com/ultralytics/yolov5/blob/74b34872fdf41941cddcf243951cdb090fbac17b/data/coco.yaml#L14) dataset (1GB - 5000 images), and test model accuracy.

In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

  0%|          | 0.00/780M [00:00<?, ?B/s]

In [ ]:
# Run YOLOv5x on COCO val
!python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5x.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-343-g685332e Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

100% 166M/166M [00:16<00:00, 10.3MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
100% 755k/755k [00:00<00:00, 14.8MB/s]
val: Scanning '/content/datasets/coco/val2017' images and labels...4952 found, 48 missing, 0 empty, 0 corrupt: 100% 5000/5000 [00:00<00:00, 11214.34it/s]
val: New cache created: /content/datasets/coco/val2017.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 157/157 [01:05<00:00,  2.39it/s]
                 all       5000      36335      0.743      0.6

## COCO test
Download [COCO test2017](https://github.com/ultralytics/yolov5/blob/74b34872fdf41941cddcf243951cdb090fbac17b/data/coco.yaml#L15) dataset (7GB - 40,000 images), to test model accuracy on test-dev set (**20,000 images, no labels**). Results are saved to a `*.json` file which should be **zipped** and submitted to the evaluation server at https://competitions.codalab.org/competitions/20794.

In [ ]:
# Download COCO test-dev2017
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017labels.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip
!f="test2017.zip" && curl http://images.cocodataset.org/zips/$f -o $f && unzip -q $f -d ../datasets/coco/images

In [ ]:
# Run YOLOv5x on COCO test
!python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half --task test

# 3. Train (Default)

In [ ]:
# Tensorboard  (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

In [ ]:
%cd /content/yolov5

/content/yolov5


#4. Train Custom Data

1.   Upload custom datasets and labels from local
2.   Selecting the right weights
3.   Writing config instructions - .yaml



In [ ]:
!mkdir data/train_data
!mkdir -p train_data/{images/{train,val},labels/{train,val}}

In [ ]:
#!unzip -h
!unzip /content/yolov5/train_data.zip

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5s.pt

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 55 --data coco128_custom.yaml --weights yolov5s.pt --cache

In [ ]:
#Test inference

!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --source /content/yolov5/data/train_data/images/train/62695d1841ee720031e3da1a.jpg

#5a. Openvino Optimization

Weights and Py Export Files

In [ ]:
!python export.py -h

In [ ]:
#!python export.py --weights /content/yolov5/yolov5s.pt --include openvino --imgsz 640 --device 0
!python export.py --weights /content/yolov5/runs/train/exp/weights/best.pt --include openvino --imgsz 640 --device 0  # export

In [ ]:
!python detect.py -h

In [ ]:
!python detect.py --weights /content/yolov5/best_openvino_model/ --source /content/yolov5/8310cbdf77af74f0dfac49b54efc4715.png --conf-thres 0.10 --device cpu --hide-conf

In [ ]:
!zip -r /content/yolov5/IR_File.zip /content/yolov5/yolov5s_openvino_model
from google.colab import files
files.download("/content/yolov5/IR_File.zip")

#5a. Openvino Optimization THE HARD WAY

The YOLOv5 repo provides a script models/export.py to export Pytorch weights with extensions *.pt to ONNX weights with extensions *.onnx.

As OpenVINO 2021.1 hasn’t fully support ONNX opset version 11, we need to revise the script models/export.py in line 69 to opset version 10:
<br><br>
***torch.onnx.export(model, img, f, verbose=False, opset_version=10, input_names=['images'],#output_names=['classes', 'boxes'] if y is None else ['output'])***
<br><br>

We need to specify the output node of the IR when we use model optimizer to convert the YOLOv5 model. There are 3 output nodes in YOLOv5. We can use Netron to visualize the YOLOv5 ONNX weights. Then we find the output nodes by searching the keyword “Transpose” in Netron.

In [ ]:
!python export.py -h

In [ ]:
!python export.py  --weights /content/yolov5/runs/train/exp/weights/best.pt  --img 640 --batch 1 --include onnx

# Environments

YOLOv5 may be run in any of the following up-to-date verified environments (with all dependencies including [CUDA](https://developer.nvidia.com/cuda)/[CUDNN](https://developer.nvidia.com/cudnn), [Python](https://www.python.org/) and [PyTorch](https://pytorch.org/) preinstalled):

- **Google Colab and Kaggle** notebooks with free GPU: <a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a> <a href="https://www.kaggle.com/ultralytics/yolov5"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
- **Google Cloud** Deep Learning VM. See [GCP Quickstart Guide](https://github.com/ultralytics/yolov5/wiki/GCP-Quickstart)
- **Amazon** Deep Learning AMI. See [AWS Quickstart Guide](https://github.com/ultralytics/yolov5/wiki/AWS-Quickstart)
- **Docker Image**. See [Docker Quickstart Guide](https://github.com/ultralytics/yolov5/wiki/Docker-Quickstart) <a href="https://hub.docker.com/r/ultralytics/yolov5"><img src="https://img.shields.io/docker/pulls/ultralytics/yolov5?logo=docker" alt="Docker Pulls"></a>
